In [ ]:
!pip install streamlit transformers huggingface_hub peft torch datetime -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
# Install necessary packages
!pip install streamlit
!pip install pyngrok

In [ ]:
%%writefile app.py



import streamlit as st
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer  # Add TextIteratorStreamer import
from threading import Thread  # Add Thread import
import re
import random
from huggingface_hub import login

HF_TOKEN = "YOUR_HF_API_KEY"  # Replace with your Hugging Face token

login(token=HF_TOKEN)
# Load model and tokenizer
@st.cache_resource
def load_model_and_tokenizer():
    model_name = "iyashnayi/SocioLens-llama-1"  # Replace with your SocioLens-llama-1 model path
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
    return model, tokenizer

model, tokenizer = load_model_and_tokenizer()

# Conversational templates for greetings
conversational_templates = {
    r"^(hi|hello|hey|greetings)(\s.*)?$": {
        "greetings": ["Hello", "Greetings"],
        "status": [
            "I am ready to assist with socio-economic analysis.",
            "I am prepared to provide data-driven insights."
        ],
        "offer": [
            "How may I support your research today?",
            "What socio-economic topic would you like to explore?"
        ],
        "combine": lambda g, s, o: f"{g}. {s} {o}"
    }
}

def try_conversational_response(user_input: str) -> str | None:
    for pattern, template in conversational_templates.items():
        if re.match(pattern, user_input.lower()):
            greeting = random.choice(template["greetings"])
            status = random.choice(template["status"])
            offer = random.choice(template["offer"])
            return template["combine"](greeting, status, offer)
    return None

def get_prompt(user_input: str) -> str:
    prompt = f"""
    ### System Message:
    You are SocioLens, a world-class research assistant specializing in socio-economic public health policy analysis. Your expertise lies in analyzing the interplay of economic, social, and health factors to provide actionable insights. Deliver responses that are clear, concise, and professional, avoiding fluff or jargon. Every answer must be grounded in logical reasoning, supported by relevant statistics, data, or credible sources when possible. If data is unavailable, acknowledge limitations transparently and provide informed estimates or qualitative analysis. Use a neutral, authoritative, and approachable tone.

    ### Few-Shot Examples:
    **User Input**: What is the impact of unemployment on public health in the U.S.?
    **Response**: Unemployment in the U.S. correlates with increased mental health issues and reduced healthcare access. A 2020 study found that a 1% rise in unemployment increases depression rates by 0.8%. Uninsured rates rise, with 10% of unemployed adults losing coverage. This strains public health systems, increasing emergency care costs by $5 billion annually. Mitigation requires targeted job programs and expanded Medicaid.

    **User Input**: How does income inequality affect life expectancy?
    **Response**: Income inequality reduces life expectancy, particularly in high-inequality regions. A 2019 study showed that a 1% increase in the Gini coefficient correlates with a 0.5-year decrease in life expectancy for lower-income groups. In the U.S., the top 1% live 10-15 years longer than the bottom 1%. Policies like progressive taxation and universal healthcare can narrow this gap.

    ### Instruction:
    For complex queries, think step by step to ensure accuracy. For all responses, provide concise answers (50-100 words) backed by stats or reasoning. If unsure, state limitations clearly. Respond to the following user input:

    **User Input**: {user_input}

    ### Response:
    """
    return prompt

def generate_text_stream(prompt: str, model, tokenizer, temperature: float = 0.0, top_p: float = 0.9, max_new_tokens: int = 512):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    generation_kwargs = {
        "input_ids": inputs.input_ids,
        "max_new_tokens": max_new_tokens,
        "temperature": max(1e-8, temperature),  # Avoid zero for stability
        "top_p": top_p,
        "do_sample": temperature > 0 or top_p < 1.0,
        "streamer": streamer,
    }

    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    full_reply = ""
    for token in streamer:
        if token:
            full_reply += token
            yield token
    yield {"full_reply": full_reply}

# Streamlit UI
st.title("SocioLens: Socio-Economic Public Health Analysis")
st.write("Ask about socio-economic factors, public health policies, or related topics.")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat history
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(f'<div class="bubble">{message["content"]}</div>', unsafe_allow_html=True)

# User input
if prompt := st.chat_input("Your question"):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(f'<div class="bubble">{prompt}</div>', unsafe_allow_html=True)

    # Check for conversational response
    conversational_response = try_conversational_response(prompt)
    if conversational_response:
        with st.chat_message("assistant"):
            st.markdown(f'<div class="bubble">{conversational_response}</div>', unsafe_allow_html=True)
            st.session_state.messages.append({"role": "assistant", "content": conversational_response})
    else:
        # Generate assistant response
        with st.chat_message("assistant"):
            with st.spinner("Thinking..."):
                response_placeholder = st.empty()
                full_reply = ""
                for token in generate_text_stream(
                    get_prompt(prompt),
                    model,
                    tokenizer,
                    temperature=0.0,
                    top_p=0.9,
                    max_new_tokens=512
                ):
                    if isinstance(token, dict):
                        full_reply = token["full_reply"]
                    else:
                        full_reply += token
                        response_placeholder.markdown(f'<div class="bubble">{full_reply}</div>', unsafe_allow_html=True)
                st.session_state.messages.append({"role": "assistant", "content": full_reply})

# Custom CSS for chat bubbles with darker colors
st.markdown("""
<style>
.bubble {
    padding: 10px;
    border-radius: 10px;
    background-color: #4a4a4a;  /* Dark gray for general bubble */
    color: #ffffff;  /* White text for readability */
}
.chat-message.user .bubble {
    background-color: #4682b4;  /* Steel blue for user messages */
    color: #ffffff;
}
.chat-message.assistant .bubble {
    background-color: #355e3b;  /* Dark green for assistant messages */
    color: #ffffff;
}
</style>
""", unsafe_allow_html=True)

Overwriting app.py


In [ ]:
from pyngrok import ngrok

# Kill all existing tunnels
ngrok.kill()


In [ ]:
# Authenticate ngrok with your token
!ngrok authtoken "YOUR_NGROK_API_KEY"  # Replace this with your actual ngrok token

# Run Streamlit app in the background and expose it via ngrok
from pyngrok import ngrok
import os

# Run Streamlit app (background execution)
os.system("streamlit run app.py &")

# Open ngrok tunnel for the Streamlit app (exposing port 8501)
public_url = ngrok.connect(8501)
print(f"Streamlit app is running at: {public_url}")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app is running at: NgrokTunnel: "https://2692-34-124-134-87.ngrok-free.app" -> "http://localhost:8501"
